# Advanced model implementation

Instead of using just models from scikit, we were looking for more advanced and powerfull solution of problems, firts of all, we tried out to implement several models like `BaggingClassifier` or `HistGradientBoostingClassifier`. 

They were good, unitl we reached `0.908` roof of the possible power, so with reworking of base models, we start using updating dataset with more advanced model

In [ ]:
import random as rn

import numpy as np
import pandas as pd
import sklearn as sk

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier,\
                             BaggingClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split

After base libs, we start to updating dataset

In [ ]:
df = pd.read_csv('https://gitlab.com/MaskedTrench/csv-database/-/raw/main/dataset.csv', sep = ',')
df = df.fillna(0)

df['rto'] = df[['rto_6','rto_7', 'rto_8', 'rto_9', 'rto_10', 'rto_11', 'rto_12']].sum(axis=1)
df = df.drop(columns=['rto_6','rto_7', 'rto_8', 'rto_9', 'rto_10', 'rto_11', 'rto_12'])
df['rto_std'] = df[['rto_std_6','rto_std_7', 'rto_std_8', 'rto_std_9', 'rto_std_10', 'rto_std_11', 'rto_std_12']].sum(axis=1)
df = df.drop(columns=['rto_std_6','rto_std_7', 'rto_std_8', 'rto_std_9', 'rto_std_10', 'rto_std_11', 'rto_std_12'])
df['cnt_checks'] = df[['cnt_checks_6','cnt_checks_7', 'cnt_checks_8', 'cnt_checks_9', 'cnt_checks_10', 'cnt_checks_11', 'cnt_checks_12']].sum(axis=1)
df = df.drop(columns=['cnt_checks_6','cnt_checks_7', 'cnt_checks_8', 'cnt_checks_9', 'cnt_checks_10', 'cnt_checks_11', 'cnt_checks_12'])

df['rto_Крупы и зерновые'] = df[['rto_6_Крупы и зерновые','rto_7_Крупы и зерновые', 'rto_8_Крупы и зерновые', 'rto_9_Крупы и зерновые', 'rto_10_Крупы и зерновые', 'rto_11_Крупы и зерновые', 'rto_12_Крупы и зерновые']].sum(axis=1)
df = df.drop(columns=['rto_6_Крупы и зерновые','rto_7_Крупы и зерновые', 'rto_8_Крупы и зерновые', 'rto_9_Крупы и зерновые', 'rto_10_Крупы и зерновые', 'rto_11_Крупы и зерновые', 'rto_12_Крупы и зерновые'])
df['rto_Мясная гастрономия'] = df[['rto_6_Мясная гастрономия','rto_7_Мясная гастрономия', 'rto_8_Мясная гастрономия', 'rto_9_Мясная гастрономия', 'rto_10_Мясная гастрономия', 'rto_11_Мясная гастрономия', 'rto_12_Мясная гастрономия']].sum(axis=1)
df = df.drop(columns=['rto_6_Мясная гастрономия','rto_7_Мясная гастрономия', 'rto_8_Мясная гастрономия', 'rto_9_Мясная гастрономия', 'rto_10_Мясная гастрономия', 'rto_11_Мясная гастрономия', 'rto_12_Мясная гастрономия'])
df['rto_Овощи - Фрукты'] = df[['rto_6_Овощи - Фрукты','rto_7_Овощи - Фрукты', 'rto_8_Овощи - Фрукты', 'rto_9_Овощи - Фрукты', 'rto_10_Овощи - Фрукты', 'rto_11_Овощи - Фрукты', 'rto_12_Овощи - Фрукты']].sum(axis=1)
df = df.drop(columns=['rto_6_Овощи - Фрукты','rto_7_Овощи - Фрукты', 'rto_8_Овощи - Фрукты', 'rto_9_Овощи - Фрукты', 'rto_10_Овощи - Фрукты', 'rto_11_Овощи - Фрукты', 'rto_12_Овощи - Фрукты'])
df['rto_Птица и изделия из птицы'] = df[['rto_6_Птица и изделия из птицы','rto_7_Птица и изделия из птицы', 'rto_8_Птица и изделия из птицы', 'rto_9_Птица и изделия из птицы', 'rto_10_Птица и изделия из птицы', 'rto_11_Птица и изделия из птицы', 'rto_12_Птица и изделия из птицы']].sum(axis=1)
df = df.drop(columns=['rto_6_Птица и изделия из птицы','rto_7_Птица и изделия из птицы', 'rto_8_Птица и изделия из птицы', 'rto_9_Птица и изделия из птицы', 'rto_10_Птица и изделия из птицы', 'rto_11_Птица и изделия из птицы', 'rto_12_Птица и изделия из птицы'])
df['rto_Рыба и рыбные изделия'] = df[['rto_6_Рыба и рыбные изделия','rto_7_Рыба и рыбные изделия', 'rto_8_Рыба и рыбные изделия', 'rto_9_Рыба и рыбные изделия', 'rto_10_Рыба и рыбные изделия', 'rto_11_Рыба и рыбные изделия', 'rto_12_Рыба и рыбные изделия']].sum(axis=1)
df = df.drop(columns=['rto_6_Рыба и рыбные изделия','rto_7_Рыба и рыбные изделия', 'rto_8_Рыба и рыбные изделия', 'rto_9_Рыба и рыбные изделия', 'rto_10_Рыба и рыбные изделия', 'rto_11_Рыба и рыбные изделия', 'rto_12_Рыба и рыбные изделия'])
df['rto_Сыры'] = df[['rto_6_Сыры','rto_7_Сыры', 'rto_8_Сыры', 'rto_9_Сыры', 'rto_10_Сыры', 'rto_11_Сыры', 'rto_12_Сыры']].sum(axis=1)
df = df.drop(columns=['rto_6_Сыры','rto_7_Сыры', 'rto_8_Сыры', 'rto_9_Сыры', 'rto_10_Сыры', 'rto_11_Сыры', 'rto_12_Сыры'])

df['rto_stddev_Крупы и зерновые'] = df[['rto_stddev_6_Крупы и зерновые','rto_stddev_7_Крупы и зерновые', 'rto_stddev_8_Крупы и зерновые', 'rto_stddev_9_Крупы и зерновые', 'rto_stddev_10_Крупы и зерновые', 'rto_stddev_11_Крупы и зерновые', 'rto_stddev_12_Крупы и зерновые']].sum(axis=1)
df = df.drop(columns=['rto_stddev_6_Крупы и зерновые','rto_stddev_7_Крупы и зерновые', 'rto_stddev_8_Крупы и зерновые', 'rto_stddev_9_Крупы и зерновые', 'rto_stddev_10_Крупы и зерновые', 'rto_stddev_11_Крупы и зерновые', 'rto_stddev_12_Крупы и зерновые'])
df['rto_stddev_Мясная гастрономия'] = df[['rto_stddev_6_Мясная гастрономия','rto_stddev_7_Мясная гастрономия', 'rto_stddev_8_Мясная гастрономия', 'rto_stddev_9_Мясная гастрономия', 'rto_stddev_10_Мясная гастрономия', 'rto_stddev_11_Мясная гастрономия', 'rto_stddev_12_Мясная гастрономия']].sum(axis=1)
df = df.drop(columns=['rto_stddev_6_Мясная гастрономия','rto_stddev_7_Мясная гастрономия', 'rto_stddev_8_Мясная гастрономия', 'rto_stddev_9_Мясная гастрономия', 'rto_stddev_10_Мясная гастрономия', 'rto_stddev_11_Мясная гастрономия', 'rto_stddev_12_Мясная гастрономия'])
df['rto_stddev_Овощи - Фрукты'] = df[['rto_stddev_6_Овощи - Фрукты','rto_stddev_7_Овощи - Фрукты', 'rto_stddev_8_Овощи - Фрукты', 'rto_stddev_9_Овощи - Фрукты', 'rto_stddev_10_Овощи - Фрукты', 'rto_stddev_11_Овощи - Фрукты', 'rto_stddev_12_Овощи - Фрукты']].sum(axis=1)
df = df.drop(columns=['rto_stddev_6_Овощи - Фрукты','rto_stddev_7_Овощи - Фрукты', 'rto_stddev_8_Овощи - Фрукты', 'rto_stddev_9_Овощи - Фрукты', 'rto_stddev_10_Овощи - Фрукты', 'rto_stddev_11_Овощи - Фрукты', 'rto_stddev_12_Овощи - Фрукты'])
df['rto_stddev_Птица и изделия из птицы'] = df[['rto_stddev_6_Птица и изделия из птицы','rto_stddev_7_Птица и изделия из птицы', 'rto_stddev_8_Птица и изделия из птицы', 'rto_stddev_9_Птица и изделия из птицы', 'rto_stddev_10_Птица и изделия из птицы', 'rto_stddev_11_Птица и изделия из птицы', 'rto_stddev_12_Птица и изделия из птицы']].sum(axis=1)
df = df.drop(columns=['rto_stddev_6_Птица и изделия из птицы','rto_stddev_7_Птица и изделия из птицы', 'rto_stddev_8_Птица и изделия из птицы', 'rto_stddev_9_Птица и изделия из птицы', 'rto_stddev_10_Птица и изделия из птицы', 'rto_stddev_11_Птица и изделия из птицы', 'rto_stddev_12_Птица и изделия из птицы'])
df['rto_stddev_Рыба и рыбные изделия'] = df[['rto_stddev_6_Рыба и рыбные изделия','rto_stddev_7_Рыба и рыбные изделия', 'rto_stddev_8_Рыба и рыбные изделия', 'rto_stddev_9_Рыба и рыбные изделия', 'rto_stddev_10_Рыба и рыбные изделия', 'rto_stddev_11_Рыба и рыбные изделия', 'rto_stddev_12_Рыба и рыбные изделия']].sum(axis=1)
df = df.drop(columns=['rto_stddev_6_Рыба и рыбные изделия','rto_stddev_7_Рыба и рыбные изделия', 'rto_stddev_8_Рыба и рыбные изделия', 'rto_stddev_9_Рыба и рыбные изделия', 'rto_stddev_10_Рыба и рыбные изделия', 'rto_stddev_11_Рыба и рыбные изделия', 'rto_stddev_12_Рыба и рыбные изделия'])
df['rto_stddev_Сыры'] = df[['rto_stddev_6_Сыры','rto_stddev_7_Сыры', 'rto_stddev_8_Сыры', 'rto_stddev_9_Сыры', 'rto_stddev_10_Сыры', 'rto_stddev_11_Сыры', 'rto_stddev_12_Сыры']].sum(axis=1)
df = df.drop(columns=['rto_stddev_6_Сыры','rto_stddev_7_Сыры', 'rto_stddev_8_Сыры', 'rto_stddev_9_Сыры', 'rto_stddev_10_Сыры', 'rto_stddev_11_Сыры', 'rto_stddev_12_Сыры'])

df['cnt_checks_Крупы и зерновые'] = df[['cnt_checks_6_Крупы и зерновые','cnt_checks_7_Крупы и зерновые', 'cnt_checks_8_Крупы и зерновые', 'cnt_checks_9_Крупы и зерновые', 'cnt_checks_10_Крупы и зерновые', 'cnt_checks_11_Крупы и зерновые', 'cnt_checks_12_Крупы и зерновые']].sum(axis=1)
df = df.drop(columns=['cnt_checks_6_Крупы и зерновые','cnt_checks_7_Крупы и зерновые', 'cnt_checks_8_Крупы и зерновые', 'cnt_checks_9_Крупы и зерновые', 'cnt_checks_10_Крупы и зерновые', 'cnt_checks_11_Крупы и зерновые', 'cnt_checks_12_Крупы и зерновые'])
df['cnt_checks_Мясная гастрономия'] = df[['cnt_checks_6_Мясная гастрономия','cnt_checks_7_Мясная гастрономия', 'cnt_checks_8_Мясная гастрономия', 'cnt_checks_9_Мясная гастрономия', 'cnt_checks_10_Мясная гастрономия', 'cnt_checks_11_Мясная гастрономия', 'cnt_checks_12_Мясная гастрономия']].sum(axis=1)
df = df.drop(columns=['cnt_checks_6_Мясная гастрономия','cnt_checks_7_Мясная гастрономия', 'cnt_checks_8_Мясная гастрономия', 'cnt_checks_9_Мясная гастрономия', 'cnt_checks_10_Мясная гастрономия', 'cnt_checks_11_Мясная гастрономия', 'cnt_checks_12_Мясная гастрономия'])
df['cnt_checks_Овощи - Фрукты'] = df[['cnt_checks_6_Овощи - Фрукты','cnt_checks_7_Овощи - Фрукты', 'cnt_checks_8_Овощи - Фрукты', 'cnt_checks_9_Овощи - Фрукты', 'cnt_checks_10_Овощи - Фрукты', 'cnt_checks_11_Овощи - Фрукты', 'cnt_checks_12_Овощи - Фрукты']].sum(axis=1)
df = df.drop(columns=['cnt_checks_6_Овощи - Фрукты','cnt_checks_7_Овощи - Фрукты', 'cnt_checks_8_Овощи - Фрукты', 'cnt_checks_9_Овощи - Фрукты', 'cnt_checks_10_Овощи - Фрукты', 'cnt_checks_11_Овощи - Фрукты', 'cnt_checks_12_Овощи - Фрукты'])
df['cnt_checks_Птица и изделия из птицы'] = df[['cnt_checks_6_Птица и изделия из птицы','cnt_checks_7_Птица и изделия из птицы', 'cnt_checks_8_Птица и изделия из птицы', 'cnt_checks_9_Птица и изделия из птицы', 'cnt_checks_10_Птица и изделия из птицы', 'cnt_checks_11_Птица и изделия из птицы', 'cnt_checks_12_Птица и изделия из птицы']].sum(axis=1)
df = df.drop(columns=['cnt_checks_6_Птица и изделия из птицы','cnt_checks_7_Птица и изделия из птицы', 'cnt_checks_8_Птица и изделия из птицы', 'cnt_checks_9_Птица и изделия из птицы', 'cnt_checks_10_Птица и изделия из птицы', 'cnt_checks_11_Птица и изделия из птицы', 'cnt_checks_12_Птица и изделия из птицы'])
df['cnt_checks_Рыба и рыбные изделия'] = df[['cnt_checks_6_Рыба и рыбные изделия','cnt_checks_7_Рыба и рыбные изделия', 'cnt_checks_8_Рыба и рыбные изделия', 'cnt_checks_9_Рыба и рыбные изделия', 'cnt_checks_10_Рыба и рыбные изделия', 'cnt_checks_11_Рыба и рыбные изделия', 'cnt_checks_12_Рыба и рыбные изделия']].sum(axis=1)
df = df.drop(columns=['cnt_checks_6_Рыба и рыбные изделия','cnt_checks_7_Рыба и рыбные изделия', 'cnt_checks_8_Рыба и рыбные изделия', 'cnt_checks_9_Рыба и рыбные изделия', 'cnt_checks_10_Рыба и рыбные изделия', 'cnt_checks_11_Рыба и рыбные изделия', 'cnt_checks_12_Рыба и рыбные изделия'])
df['cnt_checks_Сыры'] = df[['cnt_checks_6_Сыры','cnt_checks_7_Сыры', 'cnt_checks_8_Сыры', 'cnt_checks_9_Сыры', 'cnt_checks_10_Сыры', 'cnt_checks_11_Сыры', 'cnt_checks_12_Сыры']].sum(axis=1)
df = df.drop(columns=['cnt_checks_6_Сыры','cnt_checks_7_Сыры', 'cnt_checks_8_Сыры', 'cnt_checks_9_Сыры', 'cnt_checks_10_Сыры', 'cnt_checks_11_Сыры', 'cnt_checks_12_Сыры'])

df = df.drop(columns=['client_id', 'rto_stddev_Рыба и рыбные изделия', 'rto_Рыба и рыбные изделия', 'cnt_checks_Рыба и рыбные изделия'])

We updated dataset to work like sum of all coif's from table for period of time. That helped us to minify unnecessary data

## Estimates

### Inner Estimate

We decided to split dataset on several sets, so for that we used the most perspective classifier - `Bagging Classifier` 

In [ ]:
inner_estimate: list = []
counter: int = 0

print('\033[1;32mInnerEstimate creating..\033[0m')

for i in range(100):
  inner_estimate.append(
    tuple([f'in_est-{counter}', rn.choice([BaggingClassifier(), HistGradientBoostingClassifier()])])
  )
  counter += 1


InnerEstimate creating..


After that we orginized data for X and y

In [ ]:
X = df.drop(columns=['is_in_club'])
y = df['is_in_club']

And only after that we started learning them, turning whole classificators into cluster

In [ ]:
for i in range(0, 100):
  # Debug print
  print(f'--------------------\nMachine id is: {inner_estimate[i][0]}')

  # Getting specified data
  X = df.drop(columns=[
     rn.choice(['rto_Крупы и зерновые', 'rto_Мясная гастрономия',
                'rto_Овощи - Фрукты', 'rto_Птица и изделия из птицы',
                'rto_Сыры']),

     rn.choice(['rto_stddev_Крупы и зерновые', 'rto_stddev_Мясная гастрономия',
                'rto_stddev_Овощи - Фрукты', 'rto_stddev_Птица и изделия из птицы',
                'rto_stddev_Сыры']),

     rn.choice(['cnt_checks_Крупы и зерновые', 'cnt_checks_Мясная гастрономия',
                'cnt_checks_Овощи - Фрукты', 'cnt_checks_Птица и изделия из птицы', 'cnt_checks_Сыры']),
     'is_in_club'
  ])
  y = df['is_in_club']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state = 2020, stratify=y)

  # Learning them 
  inner_estimate[i][1].fit(X_train, y_train)
  y_pred = inner_estimate[i][1].predict(X_test)
  y_pred_log_reg = inner_estimate[i][1].predict_proba(X_test)

  print(f'{inner_estimate[i][0]} - Accurancy is: {accuracy_score(y_test, y_pred)}')
  print(f'{inner_estimate[i][0]} - Precision is: {precision_score(y_test, y_pred)}')
  print(f'{inner_estimate[i][0]} - Recall is: {recall_score(y_test, y_pred)}')
  print(f'{inner_estimate[i][0]} - F1-Score is: {f1_score(y_test, y_pred)}\n\n')

  counter += 1

--------------------
Machine id is: in_est-0
in_est-0 - Accurancy is: 0.9081755045830804
in_est-0 - Precision is: 0.5434782608695652
in_est-0 - Recall is: 0.008052826542116283
in_est-0 - F1-Score is: 0.01587049674654817


--------------------
Machine id is: in_est-1
in_est-1 - Accurancy is: 0.9082051206112748
in_est-1 - Precision is: 0.5595238095238095
in_est-1 - Recall is: 0.007569656949589306
in_est-1 - F1-Score is: 0.01493723184490704


--------------------
Machine id is: in_est-2
in_est-2 - Accurancy is: 0.9045327331151619
in_est-2 - Precision is: 0.3218562874251497
in_est-2 - Recall is: 0.03462715413110001
in_est-2 - F1-Score is: 0.06252726479569579


--------------------
Machine id is: in_est-3
in_est-3 - Accurancy is: 0.9044883090728703
in_est-3 - Precision is: 0.30082644628099175
in_est-3 - Recall is: 0.029312288613303268
in_est-3 - F1-Score is: 0.05341943058409158


--------------------
Machine id is: in_est-4
in_est-4 - Accurancy is: 0.9046363892138425
in_est-4 - Precision is

How does it works

After startup, code will first of all, select random columns to drop. It starts learning on different datasets. We doesn't understand how to proved to them slice of data

### Unstable estimates

For working with more fluid data We used unstable models. They working like `what if` selectors and included in main estimate in one item

In [ ]:
unstable_estimate: list = []
counter: int = 0

#
for i in range(20):
    unstable_estimate.append(tuple([f'unstable_est-{counter}', rn.choice([
       BaggingClassifier(
        n_estimators=10, n_jobs=rn.randint(1, 10),
        random_state=rn.randint(0, 40)
        ),

       HistGradientBoostingClassifier(
        learning_rate=float(rn.randint(1, 10))
        )])
    ]))

    counter += 1
    print('\033[1;33mAppended unstable_estimate\033[0m')


We dont learned them for work. Reason is that they must be so ustable, so every decision they made must be unique

## Buildin

Main estimate that will be included in main Classifier сostructs through using inner estimate and sever classifiers

In [ ]:
estimate_final:list = [
      tuple(['m_est-0', BaggingClassifier()]),
      tuple(['m_est-1', HistGradientBoostingClassifier()]),
      tuple(['m_cmx_est-1', VotingClassifier(inner_estimate.copy(), voting="soft")]),
      tuple(['m_cmx_est-2', VotingClassifier(inner_estimate.copy(), voting="soft")]),
      tuple(['m_cmx_un_est-1', VotingClassifier(unstable_estimate.copy(), voting="soft")]),
]

Preparing data

In [ ]:
X = df.drop(columns=['is_in_club'])
y = df['is_in_club']


Constructing main VotingClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 42, stratify=y)

mch = VotingClassifier(estimators=estimate_final, voting='soft')
print('\033[1;32mStating learning..\033[0m')
mch.fit(X_train, y_train)
print('\033[1;32mPredictions..\033[0m')
y_pred = mch.predict(X_test)
print('\033[1;32mProbe..\033[0m')
y_pred_log_reg = mch.predict_proba(X_test)

Stating learning..
Predictions..
Probe..


And getting accuracy

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))

Accuracy: 0.908117179594758
Precision: 0.52
Recall: 0.00697986577181208
F1-Score: 0.013774834437086093
